In [16]:
!pip install langchain openai chromadb tiktoken jq python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [18]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import JSONLoader
from langchain_core.documents import Document
from collections import defaultdict
from typing import Dict, List, Tuple, Any
import os
from dotenv import load_dotenv
import tiktoken


In [4]:
def group_by_episode_act(docs: List[Document]) -> Dict[Tuple[str, str], List[Document]]:
    """
    Group documents by (episode, act).
    """
    grouped: Dict[Tuple[str, str], List[Document]] = defaultdict(list)
    for doc in docs:
        episode = doc.metadata.get("episode", "unknown_episode")
        act = doc.metadata.get("act", "unknown_act")
        grouped[(episode, act)].append(doc)
    return grouped


def sort_group_by_utterance_start(group: List[Document]) -> List[Document]:
    """
    Sort a list of docs by utterance_start (ascending).
    Missing values are treated as 0.
    """
    return sorted(
        group,
        key=lambda d: (d.metadata.get("utterance_start")
                       if d.metadata.get("utterance_start") is not None
                       else 0.0)
    )




In [6]:
class TranscriptChunker:
    """
    Chunk This American Life–style transcripts into smaller text blocks,
    grouped by episode + act and ordered by utterance_start.

    Chunks are formed by concatenating utterances until `max_words`
    is reached, with optional overlap in terms of utterances.
    """

    def __init__(self,
                 max_words: int = 350,
                 overlap_utterances: int = 2):
        """
        Args:
            max_words: Target maximum words per chunk.
            overlap_utterances: How many utterances from the end of the
                                previous chunk to repeat at the start
                                of the next chunk.
        """
        self.max_words = max_words
        self.overlap_utterances = overlap_utterances

    def _make_chunk_document(
        self,
        episode: str,
        act: str,
        chunk_index: int,
        docs_in_chunk: List[Document]
    ) -> Document:
        """
        Create a new Document representing one chunk, with aggregated metadata.
        """
        text_parts = [d.page_content.strip() for d in docs_in_chunk if d.page_content]
        chunk_text = " ".join(text_parts)

        # Aggregate metadata
        speakers = {d.metadata.get("speaker") for d in docs_in_chunk if d.metadata.get("speaker")}
        roles = {d.metadata.get("role") for d in docs_in_chunk if d.metadata.get("role")}

        starts = [d.metadata.get("utterance_start") for d in docs_in_chunk
                  if d.metadata.get("utterance_start") is not None]
        ends = [d.metadata.get("utterance_end") for d in docs_in_chunk
                if d.metadata.get("utterance_end") is not None]

        chunk_metadata: Dict[str, Any] = {
            "episode": episode,
            "act": act,
            "chunk_index": chunk_index,
            "num_utterances": len(docs_in_chunk),
            "num_words": len(chunk_text.split()),
            "speakers": sorted(list(speakers)),
            "roles": sorted(list(roles)),
            "chunk_utterance_start": min(starts) if starts else None,
            "chunk_utterance_end": max(ends) if ends else None,
        }

        return Document(page_content=chunk_text, metadata=chunk_metadata)

    def chunk_group(self,
                    episode: str,
                    act: str,
                    docs_in_group: List[Document]) -> List[Document]:
        """
        Chunk all utterances for a single (episode, act) group.
        Assumes docs_in_group are already sorted by utterance_start.
        """
        chunks: List[Document] = []
        current_docs: List[Document] = []
        current_word_count = 0
        chunk_index = 0

        for doc in docs_in_group:
            words = doc.page_content.split()
            n_words = len(words)

            # If adding this utterance would exceed the max_words, flush current chunk
            if current_docs and (current_word_count + n_words > self.max_words):
                chunk_doc = self._make_chunk_document(
                    episode, act, chunk_index, current_docs
                )
                chunks.append(chunk_doc)
                chunk_index += 1

                # prepare next chunk, with overlap
                if self.overlap_utterances > 0:
                    overlap_docs = current_docs[-self.overlap_utterances:]
                else:
                    overlap_docs = []

                current_docs = list(overlap_docs)
                current_word_count = sum(
                    len(d.page_content.split()) for d in current_docs
                )

            # add current utterance
            current_docs.append(doc)
            current_word_count += n_words

        # flush final chunk
        if current_docs:
            chunk_doc = self._make_chunk_document(
                episode, act, chunk_index, current_docs
            )
            chunks.append(chunk_doc)

        return chunks

    def chunk_documents(self, docs: List[Document]) -> List[Document]:
        """
        High-level method: groups by (episode, act), sorts within each group,
        and returns a flat list of chunk Documents.
        """
        grouped = group_by_episode_act(docs)
        all_chunks: List[Document] = []

        for (episode, act), group_docs in grouped.items():
            sorted_docs = sort_group_by_utterance_start(group_docs)
            group_chunks = self.chunk_group(episode, act, sorted_docs)
            all_chunks.extend(group_chunks)

        return all_chunks

In [7]:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["episode"] = record.get("episode")
    metadata["role"] = record.get("role")
    metadata["speaker"] = record.get("speaker")
    metadata["act"] = record.get("act")
    metadata["utterance_start"] = record.get("utterance_start")
    metadata["utterance_end"] = record.get("utterance_end")
    
    return metadata


In [8]:
loader = JSONLoader(
    file_path="data/transcripts_full.json",
    jq_schema=".[].[]",     
    content_key="utterance",  
    metadata_func=metadata_func
)

docs = loader.load()

In [9]:
print(docs[0]
)

page_content='Joe Franklin?' metadata={'source': '/Users/matthewlu/Downloads/podcast-RAG/data/transcripts_full.json', 'seq_num': 1, 'episode': 'ep-1', 'role': 'interviewer', 'speaker': 'ira glass', 'act': 'prologue', 'utterance_start': 0.17, 'utterance_end': 0.58}


In [10]:
print(docs[1])

page_content='I'm ready.' metadata={'source': '/Users/matthewlu/Downloads/podcast-RAG/data/transcripts_full.json', 'seq_num': 2, 'episode': 'ep-1', 'role': 'subject', 'speaker': 'joe franklin', 'act': 'prologue', 'utterance_start': 0.58, 'utterance_end': 1.39}


In [11]:
#Combine docs into larger chunks
chunker = TranscriptChunker(
    max_words=350,          # you can tune this
    overlap_utterances=2    # you can tune this too
)

chunked_docs = chunker.chunk_documents(docs)

print(f"Original utterances: {len(docs)}")
print(f"Chunked docs: {len(chunked_docs)}")
print(chunked_docs[0].metadata)
print(chunked_docs[0].page_content[:400], "...")


Original utterances: 163808
Chunked docs: 28642
{'episode': 'ep-1', 'act': 'prologue', 'chunk_index': 0, 'num_utterances': 15, 'num_words': 342, 'speakers': ['ira glass', 'joe franklin'], 'roles': ['host', 'interviewer', 'subject'], 'chunk_utterance_start': 0.17, 'chunk_utterance_end': 132.75}
Joe Franklin? I'm ready. It's Ira Glass here. Oh, you're the emcee on the show, Ira. I am the emcee on the show. Yes. Oh great. Ira? I-R-A, Ira? Ira, I-R-A. Oh, great. Now hold on one second, Ira. Don't go away. Hello? [UNINTELLIGIBLE]. Call me after 3 o'clock. I have great news for you. Ira. Yes. So listen, Tony. If the phone rings, take it in the back. And then come out and tell me who it is. Ju ...


In [15]:
print(chunked_docs[1])

page_content='Nobody hearing my words right now is thinking, "Oh, man, remember that show, back when it used to be good? That show, I never missed that show back in the old days, back in the first couple years before it got so-called popular. Back when it was still good." No, actually, I think that force, that human desire to say that is so strong, to say that "I was there back when that show was good," that force is so strong, it is so basic to who we are as people that I know-- OK, what are we? We are two minutes into the program-- I know that somewhere out there, one or two of you are saying, "Oh, sure. I used to listen to that show back in the first 30 seconds, back when it used to be really good. Remember back when they used to do all that crazy stuff? When they had that guy on the phone? Remember back then?" Well, from WBEZ, in the glorious city of Chicago, Illinois. The name of this show is Your Radio Playhouse. I'm your emcee. I'm your emcee, Ira Glass. OK, the idea of this sho

In [15]:
print(len(chunked_docs))

28642


In [ ]:
# Create embeddings and vector store
embeddings = OpenAIEmbeddings(openai_api_key = os.getenv("OPENAI_API_KEY"))
vector_store = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embeddings
)



In [20]:
tokenizer = tiktoken.encoding_for_model("text-embedding-3-small")

# chunked_docs is a list of Document objects, but encode expects a string.
# We need to iterate and encode the page_content of each doc.
total_tokens = 0
for doc in chunked_docs:
    total_tokens += len(tokenizer.encode(doc.page_content))

print(f"Total tokens across {len(chunked_docs)} chunks: {total_tokens}")

Total tokens across 28642 chunks: 10993831
